Link to dataset: https://www.kaggle.com/code/egenius/starter-cert-insider-threat-cfc11c55-1/input

<h1>Imported Libraries</h1>

In [2]:
import pandas as pd

<h1>Preprocessing</h1>

<h4>Import Dataset</h4>

In [3]:
dataset = pd.read_csv('email.csv')
dataset.head(5)

,id,date,user,pc,to,cc,bcc,from,size,attachments,content
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010 07:11:45,LAP0338,PC-5758,Dean.Flynn.Hines@dtaa.com;Wade_Harrison@lockhe...,Nathaniel.Hunter.Heath@dtaa.com,NaN,Lynn.Adena.Pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010 07:12:16,MOH0273,PC-6699,Odonnell-Gage@bellsouth.net,NaN,NaN,MOH68@optonline.net,29942,0,the breaking called allied reservations former...
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010 07:13:00,LAP0338,PC-5758,Penelope_Colon@netzero.com,NaN,NaN,Lynn_A_Pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010 07:13:17,LAP0338,PC-5758,Judith_Hayden@comcast.net,NaN,NaN,Lynn_A_Pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010 07:13:28,MOH0273,PC-6699,Bond-Raymond@verizon.net;Alea_Ferrell@msn.com;...,NaN,Odonnell-Gage@bellsouth.net,MOH68@optonline.net,17319,0,this kmh october holliswood number advised unu...


<h4>Pre-preprocessing</h4>

Below we convert categorical data to numeric data and restructuring the dataset before diving in to the preprocessing of the dataset.

In [4]:
# Split the 'datetime' column into 'date' and 'time' columns
split_df = dataset['date'].str.split(' ', expand=True)
dataset['date'] = split_df[0]
dataset['time'] = split_df[1].fillna('00:00:00')  # Fill missing time values with a default time if needed
dataset.head()

,id,date,user,pc,to,cc,bcc,from,size,attachments,content,time
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010,LAP0338,PC-5758,Dean.Flynn.Hines@dtaa.com;Wade_Harrison@lockhe...,Nathaniel.Hunter.Heath@dtaa.com,NaN,Lynn.Adena.Pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...,07:11:45
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010,MOH0273,PC-6699,Odonnell-Gage@bellsouth.net,NaN,NaN,MOH68@optonline.net,29942,0,the breaking called allied reservations former...,07:12:16
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010,LAP0338,PC-5758,Penelope_Colon@netzero.com,NaN,NaN,Lynn_A_Pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...,07:13:00
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010,LAP0338,PC-5758,Judith_Hayden@comcast.net,NaN,NaN,Lynn_A_Pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...,07:13:17
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010,MOH0273,PC-6699,Bond-Raymond@verizon.net;Alea_Ferrell@msn.com;...,NaN,Odonnell-Gage@bellsouth.net,MOH68@optonline.net,17319,0,this kmh october holliswood number advised unu...,07:13:28


In [5]:
# Convert email addresses to the number of addresses in each row
dataset['to'] = dataset['to'].str.count(';') + 1
dataset.head()

,id,date,user,pc,to,cc,bcc,from,size,attachments,content,time
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010,LAP0338,PC-5758,2,Nathaniel.Hunter.Heath@dtaa.com,NaN,Lynn.Adena.Pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...,07:11:45
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010,MOH0273,PC-6699,1,NaN,NaN,MOH68@optonline.net,29942,0,the breaking called allied reservations former...,07:12:16
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010,LAP0338,PC-5758,1,NaN,NaN,Lynn_A_Pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...,07:13:00
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010,LAP0338,PC-5758,1,NaN,NaN,Lynn_A_Pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...,07:13:17
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010,MOH0273,PC-6699,3,NaN,Odonnell-Gage@bellsouth.net,MOH68@optonline.net,17319,0,this kmh october holliswood number advised unu...,07:13:28


In [6]:
# Convert email addresses to the number of addresses in each row
dataset['cc'] = dataset['cc'].apply(lambda x: x.count(';') + 1 if pd.notna(x) else 0)
dataset.head()

,id,date,user,pc,to,cc,bcc,from,size,attachments,content,time
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010,LAP0338,PC-5758,2,1,NaN,Lynn.Adena.Pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...,07:11:45
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010,MOH0273,PC-6699,1,0,NaN,MOH68@optonline.net,29942,0,the breaking called allied reservations former...,07:12:16
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010,LAP0338,PC-5758,1,0,NaN,Lynn_A_Pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...,07:13:00
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010,LAP0338,PC-5758,1,0,NaN,Lynn_A_Pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...,07:13:17
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010,MOH0273,PC-6699,3,0,Odonnell-Gage@bellsouth.net,MOH68@optonline.net,17319,0,this kmh october holliswood number advised unu...,07:13:28


In [7]:
# Convert email addresses to the number of addresses in each row
dataset['bcc'] = dataset['bcc'].apply(lambda x: x.count(';') + 1 if pd.notna(x) else 0)
dataset.head()

,id,date,user,pc,to,cc,bcc,from,size,attachments,content,time
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010,LAP0338,PC-5758,2,1,0,Lynn.Adena.Pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...,07:11:45
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010,MOH0273,PC-6699,1,0,0,MOH68@optonline.net,29942,0,the breaking called allied reservations former...,07:12:16
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010,LAP0338,PC-5758,1,0,0,Lynn_A_Pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...,07:13:00
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010,LAP0338,PC-5758,1,0,0,Lynn_A_Pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...,07:13:17
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010,MOH0273,PC-6699,3,0,1,MOH68@optonline.net,17319,0,this kmh october holliswood number advised unu...,07:13:28


In [8]:
# Convert the text in 'from' to lowercase
dataset['from'] = dataset['from'].str.lower()
dataset.head()

,id,date,user,pc,to,cc,bcc,from,size,attachments,content,time
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010,LAP0338,PC-5758,2,1,0,lynn.adena.pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...,07:11:45
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010,MOH0273,PC-6699,1,0,0,moh68@optonline.net,29942,0,the breaking called allied reservations former...,07:12:16
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010,LAP0338,PC-5758,1,0,0,lynn_a_pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...,07:13:00
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010,LAP0338,PC-5758,1,0,0,lynn_a_pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...,07:13:17
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010,MOH0273,PC-6699,3,0,1,moh68@optonline.net,17319,0,this kmh october holliswood number advised unu...,07:13:28


In [9]:
# Convert the text in 'content' to lowercase
dataset['content'] = dataset['content'].str.lower()
dataset.head()

,id,date,user,pc,to,cc,bcc,from,size,attachments,content,time
0,{R3I7-S4TX96FG-8219JWFF},01/02/2010,LAP0338,PC-5758,2,1,0,lynn.adena.pratt@dtaa.com,25830,0,middle f2 systems 4 july techniques powerful d...,07:11:45
1,{R0R9-E4GL59IK-2907OSWJ},01/02/2010,MOH0273,PC-6699,1,0,0,moh68@optonline.net,29942,0,the breaking called allied reservations former...,07:12:16
2,{G2B2-A8XY58CP-2847ZJZL},01/02/2010,LAP0338,PC-5758,1,0,0,lynn_a_pratt@earthlink.net,28780,0,slowly this uncinus winter beneath addition ex...,07:13:00
3,{A3A9-F4TH89AA-8318GFGK},01/02/2010,LAP0338,PC-5758,1,0,0,lynn_a_pratt@earthlink.net,21907,0,400 other difficult land cirrocumulus powered ...,07:13:17
4,{E8B7-C8FZ88UF-2946RUQQ},01/02/2010,MOH0273,PC-6699,3,0,1,moh68@optonline.net,17319,0,this kmh october holliswood number advised unu...,07:13:28


In [10]:
# Rearrange columns
dataset = dataset[['id', 'pc', 'user', 'date', 'time', 'from', 'to', 'cc', 'bcc', 'size', 'attachments', 'content']]
dataset.head()

,id,pc,user,date,time,from,to,cc,bcc,size,attachments,content
0,{R3I7-S4TX96FG-8219JWFF},PC-5758,LAP0338,01/02/2010,07:11:45,lynn.adena.pratt@dtaa.com,2,1,0,25830,0,middle f2 systems 4 july techniques powerful d...
1,{R0R9-E4GL59IK-2907OSWJ},PC-6699,MOH0273,01/02/2010,07:12:16,moh68@optonline.net,1,0,0,29942,0,the breaking called allied reservations former...
2,{G2B2-A8XY58CP-2847ZJZL},PC-5758,LAP0338,01/02/2010,07:13:00,lynn_a_pratt@earthlink.net,1,0,0,28780,0,slowly this uncinus winter beneath addition ex...
3,{A3A9-F4TH89AA-8318GFGK},PC-5758,LAP0338,01/02/2010,07:13:17,lynn_a_pratt@earthlink.net,1,0,0,21907,0,400 other difficult land cirrocumulus powered ...
4,{E8B7-C8FZ88UF-2946RUQQ},PC-6699,MOH0273,01/02/2010,07:13:28,moh68@optonline.net,3,0,1,17319,0,this kmh october holliswood number advised unu...


<h1>Preprocessing</h1>